In [0]:
-- Tabela com todos os campos base que vamos precisar, de forma não agrupada
WITH tb_base AS (
  SELECT 
    --Campos básicos
    distinct 
    v.idVendedor, 
    ip.idPedido, 
    ip.vlPreco, 
    pr.idProduto, 
    pr.descCategoria, 
    pr.nrTamanhoNome,
    pr.nrTamanhoDescricao,
    pr.nrFotos,
    --Campos calculados para auxiliar nos indicadores
    if(pr.nrFotos > 0, 1, 0) as flTemFoto,
    if(pr.descCategoria is not null, 1, 0) as flTemDesc

  FROM silver.olist.vendedor as v
  LEFT JOIN silver.olist.item_pedido as ip ON v.idVendedor = ip.idVendedor
  LEFT JOIN silver.olist.pedido as p ON ip.idPedido = p.idPedido
  LEFT JOIN silver.olist.produto as pr ON ip.idProduto = pr.idProduto

  WHERE 1=1
    AND DATE(p.dtPedido) <= '2017-06-30'
    AND pr.descCategoria is not null
),

-- Agrupando as features para cada vendedor (exceto as que dependem de categoria)
tbFeaturesBase AS (
  SELECT  
    idVendedor,
    COUNT(DISTINCT idProduto) as produtosDistintos,
    COUNT(DISTINCT descCategoria) as categoriasDistintas,
    AVG(nrTamanhoNome) as tamanhoMedioNome,
    AVG(nrTamanhoDescricao) as tamanhoMedioDescricao,
    AVG(nrFotos) as qtdMediaFotos,
    SUM(flTemFoto)/COUNT(idProduto) as percProdutosComFoto,
    SUM(flTemDesc)/COUNT(idProduto) as percProdutosComDescricao

  FROM tb_base
  GROUP BY ALL
),

-- Tabela com resultados por categoria de cada seller
tb_VendasCategoriasSeller AS(
  SELECT
    idVendedor,
    descCategoria,
    SUM(vlPreco) as TotalVendasCategoriaSeller,
    COUNT(DISTINCT(idPedido)) as TotalPedidosCategoriaSeller

  FROM tb_base
  GROUP BY ALL
),

-- Tabela com a top categoria de cada seller por venda e pedidos
tb_topCategoriaSeller AS(
  SELECT
    DISTINCT
    idVendedor,
    FIRST_VALUE(descCategoria) OVER(PARTITION BY idVendedor ORDER BY TotalVendasCategoriaSeller DESC) as MaiorCategoriaVenda,
    FIRST_VALUE(TotalVendasCategoriaSeller) OVER(PARTITION BY idVendedor ORDER BY TotalVendasCategoriaSeller DESC) as MaiorCategoriaTotalVenda,
    FIRST_VALUE(descCategoria) OVER(PARTITION BY idVendedor ORDER BY TotalPedidosCategoriaSeller DESC) as MaiorCategoriaPedidos,
    FIRST_VALUE(TotalPedidosCategoriaSeller) OVER(PARTITION BY idVendedor ORDER BY TotalPedidosCategoriaSeller DESC) as MaiorCategoriaTotalPedidos
  FROM tb_VendasCategoriasSeller
),

-- Criar a tabela com as top 15 categorias gerais e o total de vendas delas
tb_topCategorias AS(
  SELECT
    descCategoria,
    sum(vlPreco) as TotalVendasCategoria

  FROM tb_base
  GROUP BY descCategoria 
  ORDER BY TotalVendasCategoria DESC
  LIMIT 15 --Top 15 categorias apenas
),

-- Criar a tabela com o valor de cada seller dentro das top 15 categorias gerais
tb_topCategoriasGeralSeller AS(
  SELECT
    vsc.idVendedor,
    vsc.descCategoria,
    vsc.TotalVendasCategoriaSeller/tc.TotalVendasCategoria as shareSellerCategoria
  FROM tb_VendasCategoriasSeller vsc
  INNER JOIN tb_topCategorias tc ON tc.descCategoria = vsc.descCategoria
),

--Pivot pra ficar cada categoria em uma coluna
tb_topCategoriasGeralSellerPivot AS(
  SELECT
    *
  FROM tb_topCategoriasGeralSeller
  PIVOT(any_value(shareSellerCategoria) FOR descCategoria IN ( --databricks nao me deixou fazer subquery aqui :(
    'automotivo',
    'beleza_saude',
    'brinquedos',
    'cama_mesa_banho',
    'cool_stuff',
    'eletroportateis',
    'esporte_lazer',
    'ferramentas_jardim',
    'informatica_acessorios',
    'moveis_decoracao',
    'moveis_escritorio',
    'perfumaria',
    'relogios_presentes',
    'telefonia',
    'utilidades_domesticas')
    )  
)

--Join final com todas as tabelas. 998 sellers ao todo
SELECT
  *
FROM tbFeaturesBase AS t1
LEFT JOIN tb_topCategoriaSeller AS t2 USING (idVendedor)
LEFT JOIN tb_topCategoriasGeralSellerPivot AS t3 USING (idVendedor)
ORDER BY idVendedor